In [190]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import xml.dom.minidom as MD
import os
import glob
import matplotlib.pyplot as plt

In [191]:
"""
Li&Lim .txt input to Dataframe
"""

def createDataframeFromTxt(filepath):
    with open(filepath, 'r') as f:
        data = {}
        data["vehicleCount"], data["vehicleCapacity"], _ = f.readline().split("\t")
        _, data["depotX"], data["depotY"], _, data["depotEarliest"], data["depotLatest"], _, _, _ = f.readline().split("\t")

        df = pd.read_csv(f, sep="\t", names=["ID", "X", "Y", "CAP", "EARLIEST", "LATEST", "SERVICE", "SRC", "DEST"])
        df = df.set_index("ID")
        pickups = df[df.SRC.apply(lambda id: id == 0)]
        delivs = df[df.SRC.apply(lambda id: id != 0)]
        pickups = pickups.rename(columns={"X": "P_X", "Y": "P_Y", "EARLIEST": "P_S", "LATEST": "P_T"})

        matcher = {
            "D_X" : "X",
            "D_Y" : "Y",
            "D_S" : "EARLIEST",
            "D_T" : "LATEST"
        }
        for (newkey, oldkey) in matcher.items():
            pickups[newkey] = pickups["DEST"].apply(lambda id: delivs.loc[id][oldkey])

        # validation:
        validation = np.sum(pickups.reset_index().apply(lambda row: row.ID != delivs.loc[row.DEST].SRC, axis=1))
        assert(validation == 0)
        validation = np.sum(delivs.reset_index().apply(lambda row: row.ID != pickups.loc[row.SRC].DEST, axis=1))
        assert(validation == 0)

        pickups = (pickups
                  .reset_index()
                  .reset_index()
                  .rename(columns={"index" : "ID", "ID": "P_ID", "DEST": "D_ID"})
                  .reindex_axis([
                      "ID", "CAP", "SERVICE", 
                      "P_ID", "P_X", "P_Y", "P_S", "P_T", 
                      "D_ID", "D_X", "D_Y", "D_S", "D_T"
                  ], axis=1))

        f.close()
        return (data, pickups)

In [192]:
"""
Dataframe to XML
"""
def createXMLFromDataframe(data, dataframe):
    # Structure the XML
    problem = ET.Element('problem')                       # problem
    problemType = ET.SubElement(problem,'problemType')     # problem/problemType
    fleetSize = ET.SubElement(problemType, 'fleetSize')    # problem/problemType/fleetSize
    vehicles = ET.SubElement(problem, 'vehicles')          # problem/vehicles
    vehicleTypes = ET.SubElement(problem, 'vehicleTypes')  # problem/vehicleTypes/type
    shipments = ET.SubElement(problem, 'shipments')        # problem/shipments
    solutions = ET.SubElement(problem, 'solutions')        # problem/solutions << should be empty


    # Configuration
    fleetSize.text = 'FINITE' 

    # VehicleType:
    vehicleType = ET.SubElement(vehicleTypes, 'type')
    id = ET.SubElement(vehicleType, 'id')
    capacityDimensions = ET.SubElement(vehicleType, 'capacity-dimensions')
    dimension = ET.SubElement(capacityDimensions, 'dimension', index="0")
    dimension.text = data["vehicleCapacity"]
    costs = ET.SubElement(vehicleType, 'costs')
    fixed = ET.SubElement(costs, 'fixed')
    distance = ET.SubElement(costs, 'distance')
    time = ET.SubElement(costs, 'time')
    service = ET.SubElement(costs, 'service')
    wait = ET.SubElement(costs, 'wait')
    fixed.text, distance.text, time.text, service.text, wait.text = "0", "1", "0", "0", "0"

    for i in range(int(data["vehicleCount"]), 0, -1):
        vehicle = ET.SubElement(vehicles, 'vehicle')
        id = ET.SubElement(vehicle, "id")
        typeId = ET.SubElement(vehicle, "typeId")
        returnToDepot = ET.SubElement(vehicle, "returnToDepot")        
        id.text = "LL-vehicle-#"+str(i)
        typeId.text = "LL-vehicle"
        returnToDepot.text = "false"

        startLocation = ET.SubElement(vehicle, "startLocation")
        id = ET.SubElement(startLocation, "id")
        id.text="depot"
        coord = ET.SubElement(startLocation, "coord", x=data["depotX"], y=data["depotY"])
        endLocation = ET.SubElement(vehicle, "endLocation")
        id = ET.SubElement(endLocation, "id")
        id.text="depot"
        coord = ET.SubElement(endLocation, "coord", x=data["depotX"], y=data["depotY"])
        timeSchedule = ET.SubElement(vehicle, "timeSchedule")
        start = ET.SubElement(timeSchedule, "start")
        start.text = str(data["depotEarliest"])
        end = ET.SubElement(timeSchedule, "end")
        end.text = str(data["depotLatest"])

    def createShipmentFromSeries(series):
        shipment = ET.SubElement(shipments, 'shipment', id=str(series.ID))

        pickup = ET.SubElement(shipment, 'pickup')
        location = ET.SubElement(pickup, 'location')
        location_id = ET.SubElement(location, 'id')
        location_id.text = "[x=" + str(series.P_X) + "][y=" + str(series.P_Y) + "]"
        coord = ET.SubElement(location, 'coord', {'x': str(series.P_X), 'y': str(series.P_Y) })
        duration = ET.SubElement(pickup, "duration")
        duration.text = str(series.SERVICE)
        timeWindows = ET.SubElement(pickup, 'timeWindows')
        timeWindow = ET.SubElement(timeWindows, 'timeWindow')
        start = ET.SubElement(timeWindow, 'start')
        start.text = str(series.P_S)
        end = ET.SubElement(timeWindow, 'end')
        end.text = str(series.P_T)

        delivery = ET.SubElement(shipment, 'delivery')
        location = ET.SubElement(delivery, 'location')
        location_id = ET.SubElement(location, 'id')
        location_id.text = "[x=" + str(series.D_X) + "][y=" + str(series.D_Y) + "]"
        coord = ET.SubElement(location, 'coord', {'x': str(series.D_X), 'y': str(series.D_Y) })
        duration = ET.SubElement(delivery, "duration")
        duration.text = str(series.SERVICE)
        timeWindows = ET.SubElement(delivery, 'timeWindows')
        timeWindow = ET.SubElement(timeWindows, 'timeWindow')
        start = ET.SubElement(timeWindow, 'start')
        start.text = str(series.D_S)
        end = ET.SubElement(timeWindow, 'end')
        end.text = str(series.D_T)

        capacityDimensions = ET.SubElement(shipment, 'capacity-dimensions')
        dimension = ET.SubElement(capacityDimensions, 'dimension', index="0")
        dimension.text = str(series.CAP)

    dataframe.apply(createShipmentFromSeries, axis=1)
    dataframe = None
    data = None
    return problem
    

In [206]:
from time import sleep

for filepath in glob.glob("input/**/*.txt"):
    data, dataframe = createDataframeFromTxt(filepath)
    #     plt.scatter(np.array(dataframe.P_X), np.array(dataframe.P_Y))
    #     plt.title("Pickup points")
    #     plt.show()
    XMLRoot = createXMLFromDataframe(data, dataframe)
    s = str(ET.tostring(XMLRoot, 'utf-8'))
      
    outputfile = "output" + filepath[5:-3] + "xml"
    directory = outputfile[:outputfile.rfind('/')]
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(outputfile, 'w') as f:
        f.write(s)


